# Comparing different DFO for Privugger-AG
*MADE BY: Mathias Oliver Valdbjørn Jørgensen maoj@itu.dk*


This document contains a comparison of different derivative free optimization technique for privacy risk analysis.

# Imports

In [ ]:
import numpy as np
import opendp.smartnoise.core as sn
from sklearn.feature_selection import mutual_info_regression
from typing import List, Tuple
from scipy import stats as st
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import re
import subprocess
from entropy_estimators import continuous
from GPyOpt.methods import BayesianOptimization #Bayesian Optimization
import pybobyqa # For Bobyqa
from scipy import optimize # For Powell
import pyswarms as ps

## Random Search

In [ ]:
def random_search(f, domain, iterations=10_000):
    def val():
        v = []
        for d in domain:
            s = np.random.uniform(d["domain"][0],d["domain"][1])
            v.append(s)
        return np.array(v)
    best = np.inf
    x_best = []
    for i in range(iterations):
        x = val()
        b = f(x)
        if b < best:
            best = b
            x_best = x
    return best, x_best

## Powell

In [ ]:
def powell(f, domain):
    v = []
    for d in domain:
        v.append(np.random.uniform(d["domain"][0],d["domain"][1]))
    bounds = [d["domain"] for d in domain]
    
    return optimize.minimize(f, 
            v, 
            method="Powell", 
            bounds=bounds,
            options={'xtol': 0.0000001, 'ftol': 0.000001})

## Bobyqa

In [ ]:
def boby(f, domain):
    x0 = []
    for d in domain:
        x0.append(np.random.uniform(d["domain"][0],d["domain"][1]))
    lower_bounds = [d["domain"][0] for d in domain]
    upper_bounds = [d["domain"][1] for d in domain]
    return pybobyqa.solve(f, x0, bounds=(lower_bounds, upper_bounds))

## Particle Swarm

In [ ]:
def particle_swarm(f,domain, n_particles=30):
    lower_bounds = [d["domain"][0] for d in domain]
    upper_bounds = [d["domain"][1] for d in domain]
    bounds = (np.array(lower_bounds), np.array(upper_bounds))
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
    optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=len(domain), options=options, bounds=bounds)

    return optimizer.optimize(f, 180)

## Bayesian optimization

In [ ]:
def b_opt(f, domain):
    INITIAL_DESIGN_NUMDATA = 10
    MAX_ITER = 50
    ds = []
    for i,d in enumerate(domain):
        ds.append({"name": f"alice_{i}", "type": "continuous", "domain": d["domain"]})
    Bopt =  BayesianOptimization(f=f, domain=ds, 
                             acquisition_type='EI',        # Expected Improvement
                             initial_design_numdata=INITIAL_DESIGN_NUMDATA,
                             exact_feval = True)
    Bopt.run_optimization(max_iter = MAX_ITER-INITIAL_DESIGN_NUMDATA, eps=1e-8) 
    return (Bopt.fx_opt, Bopt.x_opt)

# Evaluations


In [ ]:
def evaluate(f, domain, best_f, best_x,eps=0.02):
    reachesMax = [False]*5
    accuracy = [0]*5
    for i in range(100):
        f1, x1 = random_search(f,domain, iterations=1_000)

        temp = powell(f,domain)
        f2 = temp.fun
        x2 = temp.x

        temp = boby(f, domain)
        f3, x3 = temp.f, temp.x
        
        f_part = lambda x: np.array([f(xi) for xi in x])
        f4, x4 = particle_swarm(f_part,domain)
        
        f_bo = lambda x: f(x[0])
        f5,x5 = b_opt(f_bo,domain)
        
        for i,res in enumerate([f1,f2,f3,f4,f5]):
            if best_f-eps <= res <= eps+best_f:
                accuracy[i] += 1
                reachesMax[i] = True
            
    return accuracy, reachesMax

## Binomial entropy

In [ ]:
def bern_entropy(x):
    p = x[0]
    return -np.log2(np.exp(st.bernoulli(p).entropy()))

maximum_x = [0.5]
maximum_f = -1

parameters_domain = [
    {"domain": (0,1)}
]
evaluate(bern_entropy, parameters_domain, maximum_f, maximum_x)

## Uniform entropy

In [ ]:
def uni_entropy(x):
    u,l = max(x), min(x)
    return -np.log2(np.exp(st.randint(l,u).entropy()))

maximum_x = [0,200]
maximum_f = np.log2(200)

parameters_domain = [
    {"domain": (0,100)},
    {"domain": (100,200)}
]
evaluate(uni_entropy, parameters_domain, maximum_f, maximum_x)

## Normal entropy

In [ ]:
def normal_MI(x):
    mu1, sigma1, mu2, sigma2 = np.abs(x)
    a = st.norm(mu1, sigma1).rvs(10_000)
    b = st.norm(mu2,sigma2).rvs(10_000)
    o = (a+b)/2
    return -mutual_info_regression(a.reshape(-1,1),o,discrete_features=False)[0]

maximum_x = [10,15,10,0.1]
maximum_f = normal_MI(maximum_x)

parameters_domain = [
    {"domain": (0,100)},
    {"domain": (0.1,15)},
    {"domain": (0,100)},
    {"domain": (0.1,15)}
]
evaluate(normal_MI, parameters_domain, maximum_f, maximum_x)